# MessageBuilder Local Paths HelloWorld Demonstration

This notebook demonstrates the **MessageBuilder** system's new local file path functionality - a convenient way to add local files directly by path instead of pre-loading them as bytes.

## Key Features

- 🔧 **Path-based Methods**: Direct local file integration with `add_local_image()`, `add_local_document()`, and `add_local_video()`
- 🧩 **Type Safety**: Enum-based formats and validation
- 🎯 **Auto-Detection**: Automatic format detection from file content and extensions
- 📝 **Simplified Usage**: No need to manually read files into bytes
- 🖼️ **Size Validation**: Built-in file size checking with configurable limits
- 📄 **Error Handling**: Comprehensive validation and error messages
- ✅ **Same API**: Consistent fluent interface with existing byte methods

## Comparison with Bytes Approach

**Bytes approach:**
```python
def read_image_bytes(image_path: str) -> bytes:
    with open(image_path, "rb") as f:
        return f.read()

image_bytes = read_image_bytes("../images/photo.jpg")
message = create_user_message() \
    .add_image_bytes(bytes=image_bytes, filename="photo.jpg") \
    .build()
```

**Path-based approach:**
```python
message = create_user_message() \
    .add_local_image(path_to_local_file="../images/photo.jpg") \
    .build()
```

## Setup and Imports

In [ ]:
import sys
import json
from pathlib import Path
import logging
from datetime import datetime

# Add the src directory to path for imports
sys.path.append(str(Path.cwd().parent / "src"))

# Import the LLMManager and related classes
from bestehorn_llmmanager.llm_manager import LLMManager
from bestehorn_llmmanager.bedrock.models.llm_manager_structures import AuthConfig, RetryConfig, AuthenticationType, RetryStrategy
from bestehorn_llmmanager.bedrock.exceptions.llm_manager_exceptions import LLMManagerError, ConfigurationError, AuthenticationError

# Import MessageBuilder components
from bestehorn_llmmanager.message_builder import create_assistant_message, create_message
from bestehorn_llmmanager.message_builder_enums import RolesEnum, ImageFormatEnum, DocumentFormatEnum, VideoFormatEnum

# Configure logging for better visibility
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ Imports successful!")
print(f"📁 Working directory: {Path.cwd()}")
print("🔧 MessageBuilder with local path support imported and ready!")

ModuleNotFoundError: No module named 'bestehorn_llmmanager.bedrock.models.message_builder_factory'

## Helper Functions

In [ ]:
def display_response(response, title="Response"):
    """Display a formatted response from LLMManager."""
    print(f"\n{title}")
    print("=" * len(title))
    
    if response.success:
        print(f"✅ Success: {response.success}")
        print(f"🤖 Model: {response.model_used}")
        print(f"🌍 Region: {response.region_used}")
        print(f"🔗 Access Method: {response.access_method_used}")
        print(f"⏱️  Total Duration: {response.total_duration_ms:.2f}ms")
        if response.api_latency_ms:
            print(f"⚡ API Latency: {response.api_latency_ms:.2f}ms")
        
        content = response.get_content()
        if content:
            print(f"\n💬 Response Content:")
            print("-" * 20)
            print(content)
        
        usage = response.get_usage()
        if usage:
            print(f"\n📊 Token Usage:")
            for key, value in usage.items():
                print(f"   {key}: {value}")
    else:
        print(f"❌ Success: {response.success}")
        print(f"🔄 Attempts: {len(response.attempts)}")
    
    if response.warnings:
        print(f"\n⚠️  Warnings:")
        for warning in response.warnings:
            print(f"   - {warning}")

def display_file_info(file_path: str, content_type: str = "file"):
    """Display information about a file."""
    path = Path(file_path)
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        print(f"📁 {content_type.title()}: {path.name}")
        print(f"   📏 Size: {size_mb:.2f} MB ({path.stat().st_size:,} bytes)")
        print(f"   📍 Path: {path}")
        return True
    else:
        print(f"❌ {content_type.title()} file not found: {file_path}")
        return False

print("✅ Helper functions defined!")

## Initialize the LLMManager

In [ ]:
print("🚀 Initializing LLMManager...")

models = ["Claude 3.5 Sonnet v2", "Claude 3 Haiku", "Nova Pro", "Nova Lite"]
regions = ["us-east-1", "us-west-2", "eu-west-1"]

auth_config = AuthConfig(auth_type=AuthenticationType.PROFILE, profile_name="default")
retry_config = RetryConfig(max_retries=3, retry_strategy=RetryStrategy.REGION_FIRST)

try:
    manager = LLMManager(models=models, regions=regions, auth_config=auth_config, retry_config=retry_config, timeout=30)
    print(f"✅ LLMManager initialized successfully!")
    validation = manager.validate_configuration()
    print(f"   Valid: {'✅' if validation['valid'] else '❌'} {validation['valid']}")
except Exception as e:
    print(f"❌ Error initializing LLMManager: {e}")
    raise

## Example 1: Local Image Analysis 🖼️
Using the new `add_local_image()` method for direct path-based image loading.

In [ ]:
print("🖼️  Example 1: Local Image Analysis (Path-based)")
print("=" * 50)

eiffel_image_path = "../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg"

if display_file_info(eiffel_image_path, "image"):
    try:
        # Build the message using the new add_local_image method
        message = create_user_message() \
            .add_text("Please analyze this image in detail. What do you see? Describe the architecture, setting, and any notable features. Also identify what landmark this is.") \
            .add_local_image(path_to_local_file=eiffel_image_path, max_size_mb=5.0) \
            .build()
        
        messages = [message]
        print(f"🔧 Built message with {len(message['content'])} content blocks using path-based method")
        print(f"   📸 Image format detected: {message['content'][1]['image']['format']}")
        
        response = manager.converse(messages=messages, inference_config={"maxTokens": 800, "temperature": 0.3})
        display_response(response, "🖼️  Local Image Analysis Response")
        
    except Exception as e:
        print(f"❌ Error in local image analysis: {e}")

## Example 2: Multi-Image Comparison with Paths 🔍
Using multiple local images with automatic format detection.

In [ ]:
print("🔍 Example 2: Multi-Image Comparison (Path-based)")
print("=" * 50)

eiffel_path = "../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg"
tokyo_path = "../images/Tokyo_Tower_2023.jpg"

eiffel_exists = display_file_info(eiffel_path, "Eiffel Tower image")
tokyo_exists = display_file_info(tokyo_path, "Tokyo Tower image")

if eiffel_exists and tokyo_exists:
    try:
        # Build the message using multiple add_local_image calls
        message = create_user_message() \
            .add_text("Please compare these two tower images. What are the similarities and differences? Identify both landmarks and discuss their architectural styles, historical significance, and visual characteristics.") \
            .add_local_image(path_to_local_file=eiffel_path, max_size_mb=5.0) \
            .add_local_image(path_to_local_file=tokyo_path, max_size_mb=5.0) \
            .build()
        
        messages = [message]
        print(f"🔧 Built message with {len(message['content'])} content blocks (text + 2 local images)")
        print(f"   📸 First image format: {message['content'][1]['image']['format']}")
        print(f"   📸 Second image format: {message['content'][2]['image']['format']}")
        
        response = manager.converse(messages=messages, inference_config={"maxTokens": 1000, "temperature": 0.4})
        display_response(response, "🔍 Multi-Image Comparison Response")
        
    except Exception as e:
        print(f"❌ Error in multi-image comparison: {e}")

## Example 3: Local Document Analysis 📄
Using the new `add_local_document()` method for direct document processing.

In [ ]:
print("📄 Example 3: Local Document Analysis (Path-based)")
print("=" * 50)

doc_path = "../docs/TheEiffelTowerPressKit-July2024.pdf"

if display_file_info(doc_path, "document"):
    try:
        # Build the message using the new add_local_document method
        message = create_user_message() \
            .add_text("Please analyze this PDF document about the Eiffel Tower. Provide a summary of the key information, including historical facts, visitor statistics, and any notable features mentioned.") \
            .add_local_document(
                path_to_local_file=doc_path,
                name="Eiffel Tower Press Kit",
                max_size_mb=5.0
            ) \
            .build()
        
        messages = [message]
        print(f"🔧 Built message with {len(message['content'])} content blocks using path-based document method")
        print(f"   📄 Document format detected: {message['content'][1]['document']['format']}")
        print(f"   📄 Document name: {message['content'][1]['document'].get('name', 'N/A')}")
        
        response = manager.converse(messages=messages, inference_config={"maxTokens": 1200, "temperature": 0.3})
        display_response(response, "📄 Local Document Analysis Response")
        
    except Exception as e:
        print(f"❌ Error in local document analysis: {e}")

## Example 4: Local Video Analysis 🎥
Using the new `add_local_video()` method for direct video processing.

In [ ]:
print("🎥 Example 4: Local Video Analysis (Path-based)")
print("=" * 45)

video_path = "../videos/EiffelTower.mp4"

if display_file_info(video_path, "video"):
    try:
        # Build the message using the new add_local_video method
        message = create_user_message() \
            .add_text("Please analyze this video of the Eiffel Tower. Describe what you see happening in the video, the perspective/camera angle, lighting conditions, any movement or activity, and the overall atmosphere. What time of day does it appear to be?") \
            .add_local_video(
                path_to_local_file=video_path,
                max_size_mb=150.0
            ) \
            .build()
        
        messages = [message]
        print(f"🔧 Built message with {len(message['content'])} content blocks using path-based video method")
        print(f"   🎥 Video format detected: {message['content'][1]['video']['format']}")
        
        response = manager.converse(messages=messages, inference_config={"maxTokens": 1000, "temperature": 0.3})
        display_response(response, "🎥 Local Video Analysis Response")
        
    except Exception as e:
        print(f"❌ Error in local video analysis: {e}")
        if "size" in str(e).lower() or "limit" in str(e).lower():
            print("💡 This might be due to video file size limits.")

## Example 5: Mixed Content with Path and Bytes Methods 🔀
Demonstrating compatibility between path-based and bytes-based methods.

In [ ]:
print("🔀 Example 5: Mixed Content (Path + Bytes Methods)")
print("=" * 48)

image_path = "../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg"
doc_path = "../docs/TheEiffelTowerPressKit-July2024.pdf"

image_exists = display_file_info(image_path, "image")
doc_exists = display_file_info(doc_path, "document")

if image_exists and doc_exists:
    try:
        # Demonstrate loading document as bytes (traditional way)
        with open(doc_path, "rb") as f:
            doc_bytes = f.read()
        
        # Build message mixing path-based image and bytes-based document
        message = create_user_message() \
            .add_text("I'm providing both an image and a document about the Eiffel Tower. Please analyze both and create a comprehensive overview combining visual and textual information.") \
            .add_local_image(path_to_local_file=image_path, max_size_mb=5.0) \
            .add_document_bytes(bytes=doc_bytes, filename=Path(doc_path).name, name="Eiffel Tower Reference") \
            .build()
        
        messages = [message]
        print(f"🔧 Built message with {len(message['content'])} content blocks using mixed methods")
        print(f"   📸 Image (path-based): {message['content'][1]['image']['format']}")
        print(f"   📄 Document (bytes-based): {message['content'][2]['document']['format']}")
        
        response = manager.converse(messages=messages, inference_config={"maxTokens": 1500, "temperature": 0.4})
        display_response(response, "🔀 Mixed Content Analysis Response")
        
    except Exception as e:
        print(f"❌ Error in mixed content analysis: {e}")

## Example 6: Error Handling and Validation 🛡️
Demonstrating error handling for invalid paths and oversized files.

In [ ]:
print("🛡️ Example 6: Error Handling and Validation")
print("=" * 45)

# Test 1: Non-existent file
print("\n🧪 Test 1: Non-existent file")
try:
    message = create_user_message() \
        .add_text("This should fail") \
        .add_local_image(path_to_local_file="../images/non_existent_file.jpg") \
        .build()
    print("❌ Expected error did not occur")
except FileNotFoundError as e:
    print(f"✅ Correctly caught FileNotFoundError: {e}")
except Exception as e:
    print(f"❓ Unexpected error type: {type(e).__name__}: {e}")

# Test 2: Directory instead of file
print("\n🧪 Test 2: Directory instead of file")
try:
    message = create_user_message() \
        .add_text("This should fail") \
        .add_local_image(path_to_local_file="../images/") \
        .build()
    print("❌ Expected error did not occur")
except Exception as e:
    print(f"✅ Correctly caught error: {type(e).__name__}: {e}")

print("\n✅ Error handling validation completed!")